# 0. Instructions and setup

## 0.1. Instructions. Part 1: Setting Up the Problem (1.5 points)

- **Objective**: Understand and establish the baseline for your chosen dataset.

- **Tasks:**
  - **a. Bibliography and SOA (0.25 points):** Present briefly your task by researching and documenting the main  objective, a potential business case and the current state of the art  for your dataset's task. Include relevant benchmarks and methodologies.  You can look at google scholar, NLP index or papers with code. 
  - **b. Dataset Description (0.5 points):** Provide a brief overview of your dataset, including size, class  distribution, and any peculiar characteristics. Include basic  descriptive statistics.
  - **c. Random Classifier Performance (0.25 points):** Calculate the expected performance of a random classifier for your  dataset to set a benchmark. The calculation should include an  implementation. 
  - **d. Baseline Implementation (0.5 points):** Develop a rule-based classifier as a baseline. Discuss its performance in the  context of the dataset's complexity and compare it with human-level  performance if available.

## 0.2. Libraries

In [1]:
from datasets import load_dataset
from pprint import pprint

## 0.3. Loading the data: Swiss Judgement Prediction

Source: https://huggingface.co/datasets/rcds/swiss_judgment_prediction

# 1. Bibliography and SOA

# 2. Dataset Description

# 3. Random Classifier Performance

# 4. Baseline Implementation